## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [4]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
# week_ending = format_date_with_ordinal(today)
# week_starting = format_date_with_ordinal(one_week_ago)

week_ending = "Nov. 24th"
week_starting = "Nov. 17th"

print(week_starting, week_ending)

Nov. 17th Nov. 24th


In [5]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [6]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [7]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [8]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [9]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,November-21-2024,Condo/Co-op,315 NE 3rd Ave #1003,Fort Lauderdale,FL,33301.0,550000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/315-...,Beaches MLS,F10447810,N,Y,26.125814,-80.141029


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [10]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [11]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
114,PAST SALE,2024-11-25,Condo/Co-op,888 Intracoastal Dr Unit 9F,Fort Lauderdale,FL,33304.0,245000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/888-...,Beaches MLS,F10415832,N,Y,26.134578,-80.109837
25,PAST SALE,2024-11-25,Condo/Co-op,3995 W Mcnab Rd Unit B107,Pompano Beach,FL,33069.0,168000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/3995-W...,MARMLS,A11652063,N,Y,26.210704,-80.176162
53,PAST SALE,2024-11-24,Condo/Co-op,3221 Sabal Palm Mnr #206,Davie,FL,33024.0,250000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3221-Sabal...,MARMLS,A11643344,N,Y,26.037362,-80.246435
88,PAST SALE,2024-11-23,Condo/Co-op,6301 N Falls Cir Dr #401,Lauderhill,FL,33319.0,180000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/6301-N-Fa...,MARMLS,A11606849,N,Y,26.176771,-80.234589
60,PAST SALE,2024-11-22,Condo/Co-op,201 S Golf Blvd #295,Pompano Beach,FL,33064.0,160000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/201-S-...,Beaches MLS,RX-11015437,N,Y,26.263884,-80.126268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,PAST SALE,2024-11-18,Condo/Co-op,3680 Inverrary Dr Unit 2-Z,Lauderhill,FL,33319.0,190000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/3680-Inve...,Beaches MLS,F10462917,N,Y,26.170807,-80.231661
44,PAST SALE,2024-11-18,Condo/Co-op,7751 Southampton Ter Unit 112h,Tamarac,FL,33321.0,215000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Tamarac/7751-Southam...,Beaches MLS,RX-11019846,N,Y,26.221521,-80.288941
89,PAST SALE,2024-11-18,Condo/Co-op,2600 S Ocean Dr Unit S-202,Hollywood,FL,33019.0,219000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2600-S-Oce...,Beaches MLS,F10440581,N,Y,25.998032,-80.118547
19,PAST SALE,2024-11-18,Condo/Co-op,1720 Harrison St Unit 12H,Hollywood,FL,33020.0,265000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1720-Harri...,MARMLS,A11675741,N,Y,26.010469,-80.141927


In [12]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [13]:
df_top_ten = df.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

## Current Week's Values

In [19]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

119


In [20]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$288,204


In [21]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$261


In [22]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$34,296,300


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [23]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
109
Input Previous Week Condo Average Sales Price:
353122
Input Previous Week Condo Average PSF:
282
Input Previous Week Condo Sales total (ex: 198_000_000)
38_500_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [24]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [25]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [26]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [27]:
pd.set_option('display.max_columns',None)

In [28]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-21,Condo/Co-op,411 N New River Dr E #1502,Fort Lauderdale,FL,33301.0,1375000.0,2.0,2.0,Las Olas Grand,1630.0,NaN,2005.0,NaN,844.0,1744.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10462851,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Elizabeth Caldwell,"Compass Florida, LLC",Michelle Dery,"Compass Florida, LLC",Rachel Buemi,D'Angelo Realty Group LLC,None,None,1,orange


In [29]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [30]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-21,Condo/Co-op,411 N New River Dr E #1502,Fort Lauderdale,FL,33301.0,1375000.0,2.0,2.0,Las Olas Grand,1630.0,NaN,2005.0,NaN,844.0,1744.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10462851,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Elizabeth Caldwell,"Compass Florida, LLC",Michelle Dery,"Compass Florida, LLC",Rachel Buemi,D'Angelo Realty Group LLC,None,None,1,orange
1,PAST SALE,2024-11-20,Condo/Co-op,1850 S Ocean Dr #3410,Hallandale Beach,FL,33009.0,1200000.0,2.0,2.0,BEACH CLUB CONDO,1782.0,NaN,2005.0,NaN,673.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11577587,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Alfred Bader,"Fathom Realty FL, LLC",None,None,Yelena Zborovsky,Exclusively Baronoff Rlty LLC,None,None,2,blue
2,PAST SALE,2024-11-21,Condo/Co-op,1012 N Ocean Blvd #908,Pompano Beach,FL,33062.0,1100000.0,3.0,3.0,Century Plaza Condo,1700.0,NaN,1971.0,NaN,647.0,1247.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1012-N...,Beaches MLS,RX-10983328,N,Y,26.247632,-80.086088,https://www.redfin.com/FL/Pompano-Beach/1012-N...,David Robertson,William Raveis Real Estate,None,None,Jake Friedman,Compass Florida LLC,None,None,3,blue
3,PAST SALE,2024-11-21,Condo/Co-op,3101 Bayshore Dr #2204,Fort Lauderdale,FL,33304.0,922500.0,2.0,2.0,FORT LAUDERDALE RESIDENCE,1176.0,NaN,2011.0,NaN,784.0,1851.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3101...,MARMLS,A11496041,N,Y,26.128581,-80.103951,https://www.redfin.com/FL/Fort-Lauderdale/3101...,Vladimir Kovrizhnykh,"Miami Homes, LLC.",Tatiana Dzenis,"Miami Homes, LLC.",None,None,None,None,4,blue
4,PAST SALE,2024-11-19,Condo/Co-op,3750 Galt Ocean Dr #711,Fort Lauderdale,FL,33308.0,850000.0,3.0,2.5,Regency Tower South,2323.0,NaN,1970.0,NaN,366.0,2111.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Beaches MLS,F10444938,N,Y,26.174073,-80.098173,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Justin Bartholomew,S & B Realty,Nancy Seltzer,S & B Realty,Mauro Rodriguez,Partnership Realty Inc.,None,None,5,blue
5,PAST SALE,2024-11-21,Condo/Co-op,4111 S Ocean Dr #3004,Hollywood,FL,33019.0,850000.0,2.0,2.0,4111 SOUTH OCEAN DRIVE CO,1126.0,NaN,2017.0,NaN,755.0,1453.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,MARMLS,A11647891,N,Y,25.986806,-80.118650,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,Diego Porras Calleja,EXP Realty LLC,None,None,Juan Morales Alfonso,"Real Estate Empire Group, Inc.",None,None,6,blue
6,PAST SALE,2024-11-21,Condo/Co-op,920 NE 16th Ter #1,Fort Lauderdale,FL,33304.0,825000.0,3.0,3.5,La Terrace,2212.0,NaN,2005.0,NaN,373.0,530.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10455169,N,Y,26.136230,-80.126510,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Hunter Taravella,Aqua Realty Services,None,None,Raquel Ells,Aqua Realty Services,Raquel Ells,Aqua Realty Services,7,blue
7,PAST SALE,2024-11-19,Condo/Co-op,818 NE 25th Ave #66,Hallandale Beach,FL,33009.0,775000.0,3.0,2.5,VENETIAN PARK II CONDO,1793.0,NaN,1977.0,NaN,432.0,811.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/818...,MARMLS,A11656558,N,Y,25.995355,-80.124373,https://www.redfin.com/FL/Hallandale-Beach/818...,Linda Phillips,Future Home Realty Inc,None,None,Mxim Madcedi,"FIP Realty Services, LLC",None,None,8,blue
8,PAST SALE,2024-11-18,Condo/Co-op,111 N Pompano Beach Blvd #610,Pompano Beach,FL,33062.0,660000.0,2.0,2.0,Sea Monarch,1600.

In [31]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [32]:
m.save('index.html')

## Data snagger

In [33]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [34]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-21,Condo/Co-op,411 N New River Dr E #1502,Fort Lauderdale,FL,33301.0,1375000.0,2.0,2.0,Las Olas Grand,1630.0,NaN,2005.0,NaN,844.0,1744.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10462851,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Elizabeth Caldwell,"Compass Florida, LLC",Michelle Dery,"Compass Florida, LLC",Rachel Buemi,D'Angelo Realty Group LLC,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [35]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_11252024


# CREATE TEMPLATE 

In [36]:
muni_set = set(df_top_ten['CITY'])

In [37]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [38]:
df_top_ten.reset_index(inplace=True,drop=True)

In [39]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [40]:
top_sale

'Las Olas Grand at 411 N New River Dr E #1502 in Fort Lauderdale'

In [41]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [42]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [43]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-21,Condo/Co-op,411 N New River Dr E #1502,Fort Lauderdale,FL,33301.0,1375000.0,2.0,2.0,Las Olas Grand,1630.0,NaN,2005.0,NaN,844.0,1744.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10462851,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Elizabeth Caldwell,"Compass Florida, LLC",Michelle Dery,"Compass Florida, LLC",Rachel Buemi,D'Angelo Realty Group LLC,None,None,1,orange,Las Olas Grand at 411 N New River Dr E #1502 i...
1,PAST SALE,2024-11-20,Condo/Co-op,1850 S Ocean Dr #3410,Hallandale Beach,FL,33009.0,1200000.0,2.0,2.0,BEACH CLUB CONDO,1782.0,NaN,2005.0,NaN,673.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11577587,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Alfred Bader,"Fathom Realty FL, LLC",None,None,Yelena Zborovsky,Exclusively Baronoff Rlty LLC,None,None,2,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #3410 in H...
2,PAST SALE,2024-11-21,Condo/Co-op,1012 N Ocean Blvd #908,Pompano Beach,FL,33062.0,1100000.0,3.0,3.0,Century Plaza Condo,1700.0,NaN,1971.0,NaN,647.0,1247.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1012-N...,Beaches MLS,RX-10983328,N,Y,26.247632,-80.086088,https://www.redfin.com/FL/Pompano-Beach/1012-N...,David Robertson,William Raveis Real Estate,None,None,Jake Friedman,Compass Florida LLC,None,None,3,blue,Century Plaza Condo at 1012 N Ocean Blvd #908 ...
3,PAST SALE,2024-11-21,Condo/Co-op,3101 Bayshore Dr #2204,Fort Lauderdale,FL,33304.0,922500.0,2.0,2.0,FORT LAUDERDALE RESIDENCE,1176.0,NaN,2011.0,NaN,784.0,1851.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3101...,MARMLS,A11496041,N,Y,26.128581,-80.103951,https://www.redfin.com/FL/Fort-Lauderdale/3101...,Vladimir Kovrizhnykh,"Miami Homes, LLC.",Tatiana Dzenis,"Miami Homes, LLC.",None,None,None,None,4,blue,FORT LAUDERDALE RESIDENCE at 3101 Bayshore Dr ...
4,PAST SALE,2024-11-19,Condo/Co-op,3750 Galt Ocean Dr #711,Fort Lauderdale,FL,33308.0,850000.0,3.0,2.5,Regency Tower South,2323.0,NaN,1970.0,NaN,366.0,2111.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Beaches MLS,F10444938,N,Y,26.174073,-80.098173,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Justin Bartholomew,S & B Realty,Nancy Seltzer,S & B Realty,Mauro Rodriguez,Partnership Realty Inc.,None,None,5,blue,Regency Tower South at 3750 Galt Ocean Dr #711...
5,PAST SALE,2024-11-21,Condo/Co-op,4111 S Ocean Dr #3004,Hollywood,FL,33019.0,850000.0,2.0,2.0,4111 SOUTH OCEAN DRIVE CO,1126.0,NaN,2017.0,NaN,755.0,1453.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,MARMLS,A11647891,N,Y,25.986806,-80.118650,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,Diego Porras Calleja,EXP Realty LLC,None,None,Juan Morales Alfonso,"Real Estate Empire Group, Inc.",None,None,6,blue,4111 SOUTH OCEAN DRIVE CO at 4111 S Ocean Dr #...
6,PAST SALE,2024-11-21,Condo/Co-op,920 NE 16th Ter #1,Fort Lauderdale,FL,33304.0,825000.0,3.0,3.5,La Terrace,2212.0,NaN,2005.0,NaN,373.0,530.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10455169,N,Y,26.136230,-80.126510,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Hunter Taravella,Aqua Realty Services,None,None,Raquel Ells,Aqua Realty Services,Raquel Ells,Aqua Realty Services,7,blue,La Terrace at 920 NE 16th Ter #1 in Fort Laude...
7,PAST SALE,2024-11-19,Condo/Co-op,818 NE 25th Ave #66,Hallandale Beach,FL,33009.0,775000.0,3.0,2.5,VENETIAN PARK II CONDO,1793.0,NaN,1977.0,NaN,432.0,811.0,Sold,NaN,NaN,https://www

In [44]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Las Olas Grand closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $550,000M to $1,375,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Hollywood, Fort Lauderdale, Pompano Beach, Hallandale Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 119 condo sales totaling $34,296,300 million from Nov. 17th to Nov. 24th. The previous week, brokers closed 109 condo sales totaling $38,500,000.

Last week’s units sold for an average of $288,204, lower than the $353,122 average price from the previous week. The average price per square foot fell t

In [45]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [46]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [47]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [48]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-21,Condo/Co-op,411 N New River Dr E #1502,Fort Lauderdale,FL,33301.0,1375000.0,2.0,2.0,Las Olas Grand,1630.0,NaN,2005.0,NaN,844.0,1744.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10462851,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Elizabeth Caldwell,"Compass Florida, LLC",Michelle Dery,"Compass Florida, LLC",Rachel Buemi,D'Angelo Realty Group LLC,None,None,1,orange,Las Olas Grand at 411 N New River Dr E #1502 i...
1,PAST SALE,2024-11-20,Condo/Co-op,1850 S Ocean Dr #3410,Hallandale Beach,FL,33009.0,1200000.0,2.0,2.0,BEACH CLUB CONDO,1782.0,NaN,2005.0,NaN,673.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11577587,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Alfred Bader,"Fathom Realty FL, LLC",None,None,Yelena Zborovsky,Exclusively Baronoff Rlty LLC,None,None,2,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #3410 in H...
2,PAST SALE,2024-11-21,Condo/Co-op,1012 N Ocean Blvd #908,Pompano Beach,FL,33062.0,1100000.0,3.0,3.0,Century Plaza Condo,1700.0,NaN,1971.0,NaN,647.0,1247.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1012-N...,Beaches MLS,RX-10983328,N,Y,26.247632,-80.086088,https://www.redfin.com/FL/Pompano-Beach/1012-N...,David Robertson,William Raveis Real Estate,None,None,Jake Friedman,Compass Florida LLC,None,None,3,blue,Century Plaza Condo at 1012 N Ocean Blvd #908 ...
3,PAST SALE,2024-11-21,Condo/Co-op,3101 Bayshore Dr #2204,Fort Lauderdale,FL,33304.0,922500.0,2.0,2.0,FORT LAUDERDALE RESIDENCE,1176.0,NaN,2011.0,NaN,784.0,1851.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3101...,MARMLS,A11496041,N,Y,26.128581,-80.103951,https://www.redfin.com/FL/Fort-Lauderdale/3101...,Vladimir Kovrizhnykh,"Miami Homes, LLC.",Tatiana Dzenis,"Miami Homes, LLC.",None,None,None,None,4,blue,FORT LAUDERDALE RESIDENCE at 3101 Bayshore Dr ...
4,PAST SALE,2024-11-19,Condo/Co-op,3750 Galt Ocean Dr #711,Fort Lauderdale,FL,33308.0,850000.0,3.0,2.5,Regency Tower South,2323.0,NaN,1970.0,NaN,366.0,2111.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Beaches MLS,F10444938,N,Y,26.174073,-80.098173,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Justin Bartholomew,S & B Realty,Nancy Seltzer,S & B Realty,Mauro Rodriguez,Partnership Realty Inc.,None,None,5,blue,Regency Tower South at 3750 Galt Ocean Dr #711...
5,PAST SALE,2024-11-21,Condo/Co-op,4111 S Ocean Dr #3004,Hollywood,FL,33019.0,850000.0,2.0,2.0,4111 SOUTH OCEAN DRIVE CO,1126.0,NaN,2017.0,NaN,755.0,1453.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,MARMLS,A11647891,N,Y,25.986806,-80.118650,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,Diego Porras Calleja,EXP Realty LLC,None,None,Juan Morales Alfonso,"Real Estate Empire Group, Inc.",None,None,6,blue,4111 SOUTH OCEAN DRIVE CO at 4111 S Ocean Dr #...
6,PAST SALE,2024-11-21,Condo/Co-op,920 NE 16th Ter #1,Fort Lauderdale,FL,33304.0,825000.0,3.0,3.5,La Terrace,2212.0,NaN,2005.0,NaN,373.0,530.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10455169,N,Y,26.136230,-80.126510,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Hunter Taravella,Aqua Realty Services,None,None,Raquel Ells,Aqua Realty Services,Raquel Ells,Aqua Realty Services,7,blue,La Terrace at 920 NE 16th Ter #1 in Fort Laude...
7,PAST SALE,2024-11-19,Condo/Co-op,818 NE 25th Ave #66,Hallandale Beach,FL,33009.0,775000.0,3.0,2.5,VENETIAN PARK II CONDO,1793.0,NaN,1977.0,NaN,432.0,811.0,Sold,NaN,NaN,https://www

In [49]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: Las Olas Grand closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $550,000M to $1,375,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Hollywood, Fort Lauderdale, Pompano Beach, Hallandale Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 119 condo sales totaling $34,296,300 million from Nov. 17th to Nov. 24th. The previous week, brokers closed 109 condo sales totaling $38,500,000.

Last week’s units sold for an average of $288,204, lower than the $353,122 average price from the previous week. The average price per square foot fell t

In [57]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Fort-Lauderdale/411-N-New-River-Dr-E-33301/unit-1502/home/42030582


In [58]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Hallandale-Beach/1850-S-Ocean-Dr-33009/unit-3410/home/42032997


In [59]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Fort-Lauderdale/315-NE-3rd-Ave-33301/unit-1003/home/42077034


In [60]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Pompano-Beach/111-N-Pompano-Beach-Blvd-33062/unit-610/home/41524538


In [54]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Fort-Lauderdale/411-N-New-River-Dr-E-33301/unit-1502/home/42030582


In [55]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Fort-Lauderdale/315-NE-3rd-Ave-33301/unit-1003/home/42077034


In [56]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-21,Condo/Co-op,411 N New River Dr E #1502,Fort Lauderdale,FL,33301.0,1375000.0,2.0,2.0,Las Olas Grand,1630.0,NaN,2005.0,NaN,844.0,1744.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10462851,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Elizabeth Caldwell,"Compass Florida, LLC",Michelle Dery,"Compass Florida, LLC",Rachel Buemi,D'Angelo Realty Group LLC,None,None,1,orange,Las Olas Grand at 411 N New River Dr E #1502 i...
1,PAST SALE,2024-11-20,Condo/Co-op,1850 S Ocean Dr #3410,Hallandale Beach,FL,33009.0,1200000.0,2.0,2.0,BEACH CLUB CONDO,1782.0,NaN,2005.0,NaN,673.0,1750.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11577587,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,Alfred Bader,"Fathom Realty FL, LLC",None,None,Yelena Zborovsky,Exclusively Baronoff Rlty LLC,None,None,2,blue,BEACH CLUB CONDO at 1850 S Ocean Dr #3410 in H...
2,PAST SALE,2024-11-21,Condo/Co-op,1012 N Ocean Blvd #908,Pompano Beach,FL,33062.0,1100000.0,3.0,3.0,Century Plaza Condo,1700.0,NaN,1971.0,NaN,647.0,1247.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1012-N...,Beaches MLS,RX-10983328,N,Y,26.247632,-80.086088,https://www.redfin.com/FL/Pompano-Beach/1012-N...,David Robertson,William Raveis Real Estate,None,None,Jake Friedman,Compass Florida LLC,None,None,3,blue,Century Plaza Condo at 1012 N Ocean Blvd #908 ...
3,PAST SALE,2024-11-21,Condo/Co-op,3101 Bayshore Dr #2204,Fort Lauderdale,FL,33304.0,922500.0,2.0,2.0,FORT LAUDERDALE RESIDENCE,1176.0,NaN,2011.0,NaN,784.0,1851.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3101...,MARMLS,A11496041,N,Y,26.128581,-80.103951,https://www.redfin.com/FL/Fort-Lauderdale/3101...,Vladimir Kovrizhnykh,"Miami Homes, LLC.",Tatiana Dzenis,"Miami Homes, LLC.",None,None,None,None,4,blue,FORT LAUDERDALE RESIDENCE at 3101 Bayshore Dr ...
4,PAST SALE,2024-11-19,Condo/Co-op,3750 Galt Ocean Dr #711,Fort Lauderdale,FL,33308.0,850000.0,3.0,2.5,Regency Tower South,2323.0,NaN,1970.0,NaN,366.0,2111.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Beaches MLS,F10444938,N,Y,26.174073,-80.098173,https://www.redfin.com/FL/Fort-Lauderdale/3750...,Justin Bartholomew,S & B Realty,Nancy Seltzer,S & B Realty,Mauro Rodriguez,Partnership Realty Inc.,None,None,5,blue,Regency Tower South at 3750 Galt Ocean Dr #711...
5,PAST SALE,2024-11-21,Condo/Co-op,4111 S Ocean Dr #3004,Hollywood,FL,33019.0,850000.0,2.0,2.0,4111 SOUTH OCEAN DRIVE CO,1126.0,NaN,2017.0,NaN,755.0,1453.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,MARMLS,A11647891,N,Y,25.986806,-80.118650,https://www.redfin.com/FL/Hollywood/4111-S-Oce...,Diego Porras Calleja,EXP Realty LLC,None,None,Juan Morales Alfonso,"Real Estate Empire Group, Inc.",None,None,6,blue,4111 SOUTH OCEAN DRIVE CO at 4111 S Ocean Dr #...
6,PAST SALE,2024-11-21,Condo/Co-op,920 NE 16th Ter #1,Fort Lauderdale,FL,33304.0,825000.0,3.0,3.5,La Terrace,2212.0,NaN,2005.0,NaN,373.0,530.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Beaches MLS,F10455169,N,Y,26.136230,-80.126510,https://www.redfin.com/FL/Fort-Lauderdale/920-...,Hunter Taravella,Aqua Realty Services,None,None,Raquel Ells,Aqua Realty Services,Raquel Ells,Aqua Realty Services,7,blue,La Terrace at 920 NE 16th Ter #1 in Fort Laude...
7,PAST SALE,2024-11-19,Condo/Co-op,818 NE 25th Ave #66,Hallandale Beach,FL,33009.0,775000.0,3.0,2.5,VENETIAN PARK II CONDO,1793.0,NaN,1977.0,NaN,432.0,811.0,Sold,NaN,NaN,https://www

## Time on Market Calculator

In [62]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 4, 26) ## List (Earlier) date
date2 = datetime(2024, 11, 20) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

208


## Clean CSV for Datawrapper Chart

In [63]:
chart_df = df_top_ten

In [64]:
chart_df = chart_df.fillna(" ")

In [65]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [66]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [67]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [68]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [69]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [70]:
csv_date_string = today.strftime("%m_%d_%Y")

In [71]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [72]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-11-21,411 N New River Dr E #1502 in Fort Lauderdale,"$1,375,000",2,2.0,"1,630",2005,$844,Beaches MLS,F10462851,"Elizabeth Caldwell Compass Florida, LLC Michel...",Rachel Buemi D'Angelo Realty Group LLC
1,2024-11-20,1850 S Ocean Dr #3410 in Hallandale Beach,"$1,200,000",2,2.0,"1,782",2005,$673,MARMLS,A11577587,"Alfred Bader Fathom Realty FL, LLC",Yelena Zborovsky Exclusively Baronoff Rlty LLC...
2,2024-11-21,1012 N Ocean Blvd #908 in Pompano Beach,"$1,100,000",3,3.0,"1,700",1971,$647,Beaches MLS,RX-10983328,David Robertson William Raveis Real Estate,Jake Friedman Compass Florida LLC
3,2024-11-21,3101 Bayshore Dr #2204 in Fort Lauderdale,"$922,500",2,2.0,"1,176",2011,$784,MARMLS,A11496041,"Vladimir Kovrizhnykh Miami Homes, LLC. Tatiana...",
4,2024-11-19,3750 Galt Ocean Dr #711 in Fort Lauderdale,"$850,000",3,2.5,"2,323",1970,$366,Beaches MLS,F10444938,Justin Bartholomew S & B Realty Nancy Seltzer ...,Mauro Rodriguez Partnership Realty Inc.
5,2024-11-21,4111 S Ocean Dr #3004 in Hollywood,"$850,000",2,2.0,"1,126",2017,$755,MARMLS,A11647891,Diego Porras Calleja EXP Realty LLC,"Juan Morales Alfonso Real Estate Empire Group,..."
6,2024-11-21,920 NE 16th Ter #1 in Fort Lauderdale,"$825,000",3,3.5,"2,212",2005,$373,Beaches MLS,F10455169,Hunter Taravella Aqua Realty Services,Raquel Ells Aqua Realty Services Raquel Ells A...
7,2024-11-19,818 NE 25th Ave #66 in Hallandale Beach,"$775,000",3,2.5,"1,793",1977,$432,MARMLS,A11656558,Linda Phillips Future Home Realty Inc,"Mxim Madcedi FIP Realty Services, LLC"
8,2024-11-18,111 N Pompano Beach Blvd #610 in Pompano Beach,"$660,000",2,2.0,"1,600",1970,$413,Beaches MLS,F10458832,Dennis Miller Better Homes & Gdns RE Fla 1st,"Lourdes Maestres Compass Florida, LLC Amy Kran..."
9,2024-11-21,315 NE 3rd Ave #1003 in Fort Lauderdale,"$550,000",2,2.5,"1,568",2007,$351,Beaches MLS,F10447810,Carlos Mahone Redfin Corporation,Bettina Debrabander LPT Realty
